In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [20]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile1)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile2)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [21]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [22]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [23]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [24]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5 = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    segment5['upgrade_cost_usd'] = segment5['upgrade_cost_usd']/segment5['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    segment5 = segment5.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    segment5['upgrade_cost_usd'] = segment5['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    print('segment5: upgrade cost (usd) per dwelling unit')
    display(segment5['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'], segment5['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [25]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    segment5_agg[indicator] = segment5_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    print('segment5')
    display(segment5_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Annual Total Savings

In [26]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    segment5_agg[indicator] = segment5_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    print('segment5')
    display(segment5_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [27]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    segment5_agg = segment5_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    segment5_agg[indicator] = segment5_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    print('segment5')
    display(segment5_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator],segment5_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [28]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [29]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    segment5 = results.loc[results['segment'] == 'segment5']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    segment5_agg = segment5.groupby(segment5['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    segment5_agg['baseline_energy_burden_2023_cbills.%'] = segment5_agg['baseline_energy_burden_2023_cbills.%']/segment5_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    segment5_agg[indicator] = segment5_agg[indicator]/segment5_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    segment5_agg['energy_burden_diff'] = segment5_agg['baseline_energy_burden_2023_cbills.%'] - segment5_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment5_agg = segment5_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    segment5_agg = segment5_agg[segment5_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment5_agg = segment5_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment5_agg['before'] = segment5_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    segment5_agg['after'] = segment5_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    print('segment5')
    display(segment5_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']],segment5_agg[['before', 'after']])

In [30]:
community = 'duluth'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [31]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Basic Enclosure                                                     2940.3
Heat Pump Water Heater                                              3194.7
Enhanced Enclosure                                                  4934.1
High Efficiency Heat Pump with Electric Heat Backup                31202.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34316.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34669.7
Basic Enclosure + High Efficiency Whole Home Electrification       38326.7
High Efficiency Whole Home Electrification                         38404.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    38679.8
Mininum Effi

## 1.2 Total Annual Savings

In [32]:
figure_total_savings_1(community_results)

In [33]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [34]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [35]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                      35144.658339
Mininum Efficiency Heat Pump with Electric Heat Backup             39762.000000
Electric Clothes Dryer                                              5914.743706
High Efficiency Heat Pump with Electric Heat Backup                39762.000000
Electric Cooking                                                   12722.230006
Induction Cooking                                                  39329.804348
Heat Pump Clothes Dryer                                            32436.993494
Mininum Efficiency Heat Pump with Existing Heat Backup             37835.613214
Heat Pump Water Heater                                             39724.904866
High Efficiency Whole Home Electrification                         39762.000000
Basic Enclosure                                                    34337.157808
Enhanced Enclosure                                                 35606.210876
Basic Enclosure + HPWH + Hi

In [36]:
metric = "saving_cbill.total_usd"
print("Community summary:")
results = get_summary(community_results, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
saving_results['whole_community_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
saving_results['whole_community_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
saving_results['whole_community_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
saving_results['whole_community_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
results

Community summary:


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,465.684924,-295.499443,133.299157,852.751682,1411.764671
Basic Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,433.360941,-355.166220,110.445932,827.259515,1404.318792
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,423.408988,-365.464861,79.137618,798.868206,1361.096984
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,391.085005,-416.141573,52.265525,785.630343,1336.697142
Enhanced Enclosure,1088,35606.210876,316.589857,21.119755,73.587746,447.980869,725.293165
Basic Enclosure,1042,34337.157808,242.272088,15.646708,58.845050,331.121018,572.522000
High Efficiency Whole Home Electrification,1217,39762.000000,190.048178,-743.206106,-201.148164,680.398374,1244.629084
Heat Pump Water Heater,1214,39724.904866,53.876087,-6.075560,5.846410,79.370753,161.265384
Mininum Efficiency Heat Pump with Existing Heat Backup,1159,37835.613214,34.668104,-336.351898,-133.670452,162.489206,489.092714


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renter, all income  

Segment 2. Occupied, Single family detached, pre 1940, owner, all income  

Segment 3. Occupied, Multifamily with 2-4 units, 1940-1979, all income  

Segment 4. Occupied,  Multifamily with 5+ units, 1940-1979, all income 

Segment 5. Occupied,  all building types, 1980+, all income


In [37]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']

seg5_vintage = ['1980s', '1990s', '2000s', '2010s']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure),
                     'segment'] = 'segment4'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.vintage'].isin(seg5_vintage),
                     'segment'] = 'segment5'

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2605310684.py:22: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segment1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [38]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure,18,611.385070,555.137527,113.018631,324.255896,803.572699,1086.039782
Enhanced Enclosure + High Efficiency Whole Home Electrification,18,611.385070,511.015416,-570.983138,30.380115,1185.683853,1487.929802
Basic Enclosure + High Efficiency Whole Home Electrification,18,611.385070,486.607147,-570.983138,30.380115,1060.650040,1487.929802
Basic Enclosure,18,611.385070,464.058172,110.226941,181.180736,721.641103,873.595730
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,18,611.385070,446.102644,-639.919795,-23.802398,1154.557390,1462.250420
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,18,611.385070,421.694375,-639.919795,-23.802398,946.580237,1462.250420
Heat Pump Water Heater,18,611.385070,95.289198,-11.175884,8.522537,106.382733,350.493451
Heat Pump Clothes Dryer,18,611.385070,17.530022,0.682104,1.421475,28.150964,37.881666
Induction Cooking,18,611.385070,1.844012,-3.001796,-2.020254,3.907922,4.864591


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,132,3978.602628,635.420238,-1241.576850,-41.944284,1292.570698,2398.324772
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,132,3978.602628,575.538117,-1244.022708,-106.987541,1201.946351,2324.710254
Enhanced Enclosure,130,3893.473182,550.989745,53.863218,215.776671,844.050988,1239.379671
Basic Enclosure + High Efficiency Whole Home Electrification,132,3978.602628,517.355257,-1615.099424,-211.690408,1292.570698,2395.674525
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,132,3978.602628,457.473136,-1709.112182,-248.215574,1201.946351,2324.710254
Basic Enclosure,128,3828.877313,405.891512,29.904052,149.269294,574.388400,871.891794
Heat Pump Water Heater,132,3978.602628,73.214354,-7.816238,4.544327,133.162876,235.592853
Mininum Efficiency Heat Pump with Existing Heat Backup,124,3764.614372,69.423654,-458.750688,-183.506739,309.146038,709.210761
Heat Pump Clothes Dryer,130,3914.559439,21.224526,0.879408,12.899816,30.762990,45.268752


Summary for segment: segment3: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,17,1336.204891,700.329119,311.967230,440.371251,849.143289,1218.033997
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,17,1336.204891,655.026335,224.669703,399.503097,789.080974,1197.217994
Basic Enclosure + High Efficiency Whole Home Electrification,17,1336.204891,639.728873,144.847932,359.189694,849.143289,1191.774271
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,17,1336.204891,594.426089,46.918689,281.982238,789.080974,1170.958268
High Efficiency Whole Home Electrification,17,1336.204891,462.634589,-207.733205,255.164524,676.816589,937.869644
High Efficiency Heat Pump with Electric Heat Backup,17,1336.204891,264.466762,-413.266049,126.721412,578.111677,705.414080
Enhanced Enclosure,17,1336.204891,233.057991,58.311017,95.225528,230.019607,534.258242
Basic Enclosure,17,1336.204891,186.733234,58.311017,65.687364,176.970789,435.950343
Mininum Efficiency Heat Pump with Existing Heat Backup,17,1336.204891,103.528079,-189.996797,24.177459,268.844611,350.656122


Summary for segment: segment4: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,53,2185.367208,545.663241,131.575586,282.215280,800.626894,1156.397278
Basic Enclosure + High Efficiency Whole Home Electrification,53,2185.367208,528.620778,77.263042,236.028456,800.626894,1133.697749
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,53,2185.367208,518.845964,74.144597,263.987890,797.441452,1107.632534
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,53,2185.367208,501.803501,70.989492,229.966048,797.441452,1095.755467
High Efficiency Whole Home Electrification,53,2185.367208,458.866638,42.915786,204.545362,651.143966,968.570705
High Efficiency Heat Pump with Electric Heat Backup,53,2185.367208,324.727788,-5.541896,58.127683,578.407233,785.054072
Mininum Efficiency Heat Pump with Existing Heat Backup,52,1969.269382,202.567031,-16.868784,22.634013,370.791981,602.372888
Enhanced Enclosure,45,1718.819973,199.194836,55.783870,75.125229,289.292948,415.954552
Basic Enclosure,43,1628.779212,169.404835,39.711089,64.980741,231.084747,352.733560


Summary for segment: segment5: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,365,11525.652627,386.490898,-167.952468,139.149347,634.156216,1100.063783
Basic Enclosure + High Efficiency Whole Home Electrification,365,11525.652627,364.476213,-203.455975,118.243806,634.156216,1080.339300
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,365,11525.652627,337.268620,-216.212329,83.159422,585.371543,1079.859704
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,365,11525.652627,315.253935,-278.884077,72.127559,568.644125,1076.918264
High Efficiency Whole Home Electrification,365,11525.652627,218.567786,-493.105425,-61.058625,548.465075,1075.898859
Enhanced Enclosure,280,8976.701022,194.835770,16.608145,47.602866,309.759240,448.008307
Basic Enclosure,272,8668.653250,178.756804,17.230275,45.392292,231.755926,456.701016
Heat Pump Water Heater,364,11520.507440,49.390448,-12.729203,5.047988,76.441469,135.201529
High Efficiency Heat Pump with Electric Heat Backup,365,11525.652627,28.243098,-784.148736,-287.604168,354.337828,870.597801


## 2.1 Upgrade Cost Per Dwelling Unit 

In [39]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]
saving_results[f'segment5_{income}_upgrade_cost_mean_usd'] = results[4]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2963.2
Basic Enclosure                                                     5060.7
Enhanced Enclosure                                                  6933.3
High Efficiency Heat Pump with Electric Heat Backup                32728.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35349.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35550.2
Basic Enclosure + High Efficiency Whole Home Electrification       39849.6
Enhanced Enclosure + High Efficiency Whole Home Electrification    40050.8
High Efficiency Whole Home Electrification                         40192.1
Mininum Effi

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     1127.3
Electric Clothes Dryer                                               1266.0
Induction Cooking                                                    1953.1
Heat Pump Clothes Dryer                                              2547.5
Heat Pump Water Heater                                               3307.0
Basic Enclosure                                                      4434.1
Enhanced Enclosure                                                   7439.6
High Efficiency Heat Pump with Electric Heat Backup                 43301.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         44602.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      45175.5
Basic Enclosure + High Efficiency Whole Home Electrification        49062.3
Enhanced Enclosure + High Efficiency Whole Home Electrification     49635.0
High Efficiency Whole Home Electrification                          51067.6

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      778.6
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2277.3
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                23638.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25984.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27395.0
Basic Enclosure + High Efficiency Whole Home Electrification       29682.7
High Efficiency Whole Home Electrification                         30241.8
Enhanced Enclosure + High Efficiency Whole Home Electrification    31093.2
Mininum Efficiency Heat Pump with Electric Heat Backup             38981.8
Mininum Effi

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      708.9
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2244.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                20402.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22129.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22970.4
Basic Enclosure + High Efficiency Whole Home Electrification       24572.6
Mininum Efficiency Whole Home Electrification                      25374.0
Enhanced Enclosure + High Efficiency Whole Home Electrification    25413.4
High Efficiency Whole Home Electrification                         25750.1
Mininum Effi

segment5: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Basic Enclosure                                                     2629.8
Enhanced Enclosure                                                  3133.2
Heat Pump Water Heater                                              3196.0
High Efficiency Heat Pump with Electric Heat Backup                26211.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30316.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30502.3
High Efficiency Whole Home Electrification                         33283.7
Basic Enclosure + High Efficiency Whole Home Electrification       34194.0
Enhanced Enclosure + High Efficiency Whole Home Electrification    34379.6
Mininum Effi

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [40]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_per_dollar_invested_usd'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.09170
Enhanced Enclosure                                                 0.08007
Heat Pump Water Heater                                             0.03216
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01276
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01255
Basic Enclosure + High Efficiency Whole Home Electrification       0.01221
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01193
Heat Pump Clothes Dryer                                            0.00688
Induction Cooking                                                  0.00094
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00093
High Efficiency Whole Home Electrification                        -0.00368
Electric Cooking                                                  -0.00526
High Efficiency Heat Pump with Electric Heat Backup               -0.01331
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.09154
Enhanced Enclosure                                                 0.07406
Heat Pump Water Heater                                             0.02214
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01280
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01274
Basic Enclosure + High Efficiency Whole Home Electrification       0.01054
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01026
Heat Pump Clothes Dryer                                            0.00833
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00066
Induction Cooking                                                  0.00062
High Efficiency Whole Home Electrification                        -0.00020
Electric Cooking                                                  -0.00608
High Efficiency Heat Pump with Electric Heat Backup               -0.00649
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.23982
Enhanced Enclosure                                                 0.10234
Heat Pump Water Heater                                             0.02465
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02391
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02288
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02252
Basic Enclosure + High Efficiency Whole Home Electrification       0.02155
High Efficiency Whole Home Electrification                         0.01530
High Efficiency Heat Pump with Electric Heat Backup                0.01119
Heat Pump Clothes Dryer                                            0.00722
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00266
Induction Cooking                                                  0.00062
Mininum Efficiency Whole Home Electrification                     -0.00092
Mininum Effi

segment4


upgrade_name
Basic Enclosure                                                    0.23895
Enhanced Enclosure                                                 0.08876
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02268
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02259
Basic Enclosure + High Efficiency Whole Home Electrification       0.02151
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02147
Heat Pump Water Heater                                             0.01912
High Efficiency Whole Home Electrification                         0.01782
High Efficiency Heat Pump with Electric Heat Backup                0.01592
Heat Pump Clothes Dryer                                            0.00728
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00375
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00292
Mininum Efficiency Whole Home Electrification                      0.00221
Induction Co

segment5


upgrade_name
Basic Enclosure                                                    0.06797
Enhanced Enclosure                                                 0.06218
Heat Pump Water Heater                                             0.01545
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01124
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01106
Basic Enclosure + High Efficiency Whole Home Electrification       0.01066
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01040
Heat Pump Clothes Dryer                                            0.00814
High Efficiency Whole Home Electrification                         0.00657
High Efficiency Heat Pump with Electric Heat Backup                0.00108
Induction Cooking                                                  0.00035
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00034
Electric Cooking                                                  -0.00780
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [41]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.01155
Enhanced Enclosure                                                 0.00969
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00387
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00386
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00348
Basic Enclosure + High Efficiency Whole Home Electrification       0.00347
High Efficiency Heat Pump with Electric Heat Backup                0.00287
High Efficiency Whole Home Electrification                         0.00280
Heat Pump Water Heater                                             0.00265
Mininum Efficiency Whole Home Electrification                      0.00135
Electric Cooking                                                   0.00135
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00108
Heat Pump Clothes Dryer                                            0.00047
Electric Clo

segment2


upgrade_name
Basic Enclosure                                                    0.01193
Enhanced Enclosure                                                 0.00933
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00388
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00382
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00357
Basic Enclosure + High Efficiency Whole Home Electrification       0.00351
High Efficiency Heat Pump with Electric Heat Backup                0.00306
High Efficiency Whole Home Electrification                         0.00295
Heat Pump Water Heater                                             0.00230
Electric Cooking                                                   0.00135
Mininum Efficiency Whole Home Electrification                      0.00085
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00072
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00051
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.01555
Enhanced Enclosure                                                 0.00678
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00204
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00203
Basic Enclosure + High Efficiency Whole Home Electrification       0.00182
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00182
Heat Pump Water Heater                                             0.00159
High Efficiency Heat Pump with Electric Heat Backup                0.00157
High Efficiency Whole Home Electrification                         0.00155
Electric Cooking                                                   0.00121
Mininum Efficiency Whole Home Electrification                      0.00084
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00062
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00049
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.01331
Enhanced Enclosure                                                 0.00514
Heat Pump Water Heater                                             0.00147
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00145
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00143
Basic Enclosure + High Efficiency Whole Home Electrification       0.00133
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00131
Electric Cooking                                                   0.00117
High Efficiency Whole Home Electrification                         0.00116
High Efficiency Heat Pump with Electric Heat Backup                0.00109
Mininum Efficiency Whole Home Electrification                      0.00076
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00030
Heat Pump Clothes Dryer                                            0.00030
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.00653
Enhanced Enclosure                                                 0.00588
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00235
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00233
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00213
Basic Enclosure + High Efficiency Whole Home Electrification       0.00211
Heat Pump Water Heater                                             0.00205
High Efficiency Whole Home Electrification                         0.00199
High Efficiency Heat Pump with Electric Heat Backup                0.00199
Electric Cooking                                                   0.00147
Mininum Efficiency Whole Home Electrification                      0.00129
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00066
Heat Pump Clothes Dryer                                            0.00045
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [42]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.81148
Enhanced Enclosure                                                 0.67356
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.30780
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.30716
Basic Enclosure + High Efficiency Whole Home Electrification       0.27582
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.27542
High Efficiency Heat Pump with Electric Heat Backup                0.27000
High Efficiency Whole Home Electrification                         0.24896
Heat Pump Water Heater                                             0.15865
Mininum Efficiency Whole Home Electrification                      0.14890
Electric Cooking                                                   0.12946
Mininum Efficiency Heat Pump with Electric Heat Backup             0.12655
Electric Clothes Dryer                                             0.08077
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.84282
Enhanced Enclosure                                                 0.65363
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.30720
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.30696
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.28154
Basic Enclosure + High Efficiency Whole Home Electrification       0.28102
High Efficiency Heat Pump with Electric Heat Backup                0.27227
High Efficiency Whole Home Electrification                         0.25386
Heat Pump Water Heater                                             0.15307
Electric Cooking                                                   0.13058
Mininum Efficiency Whole Home Electrification                      0.08699
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07599
Electric Clothes Dryer                                             0.06990
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.86053
Enhanced Enclosure                                                 0.37974
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13109
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.12787
Basic Enclosure + High Efficiency Whole Home Electrification       0.11681
Electric Cooking                                                   0.11674
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.11462
High Efficiency Heat Pump with Electric Heat Backup                0.11062
High Efficiency Whole Home Electrification                         0.10538
Heat Pump Water Heater                                             0.09150
Mininum Efficiency Whole Home Electrification                      0.07668
Mininum Efficiency Heat Pump with Electric Heat Backup             0.05470
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03664
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.66894
Enhanced Enclosure                                                 0.26478
Electric Cooking                                                   0.12003
Heat Pump Water Heater                                             0.09554
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.08159
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07975
Basic Enclosure + High Efficiency Whole Home Electrification       0.07535
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.07389
High Efficiency Whole Home Electrification                         0.06769
Mininum Efficiency Whole Home Electrification                      0.06642
High Efficiency Heat Pump with Electric Heat Backup                0.06233
Induction Cooking                                                  0.02228
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01997
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.42107
Enhanced Enclosure                                                 0.37726
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.17777
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.17772
High Efficiency Heat Pump with Electric Heat Backup                0.16377
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.16046
Basic Enclosure + High Efficiency Whole Home Electrification       0.16032
High Efficiency Whole Home Electrification                         0.15783
Electric Cooking                                                   0.14685
Heat Pump Water Heater                                             0.14663
Mininum Efficiency Whole Home Electrification                      0.14022
Electric Clothes Dryer                                             0.07859
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07149
Induction Co

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [43]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_total_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                 339402.8
Enhanced Enclosure + High Efficiency Whole Home Electrification    312427.2
Basic Enclosure + High Efficiency Whole Home Electrification       297504.3
Basic Enclosure                                                    283718.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     272740.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        257817.6
Heat Pump Water Heater                                              58258.4
Heat Pump Clothes Dryer                                             10717.6
Induction Cooking                                                    1127.4
Electric Cooking                                                    -1000.4
Electric Clothes Dryer                                              -4563.2
Mininum Efficiency Heat Pump with Existing Heat Backup             -32068.4
High Efficiency Whole Home Electrification                         -90484.1

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2528084.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2289837.5
Enhanced Enclosure                                                 2145263.8
Basic Enclosure + High Efficiency Whole Home Electrification       2058351.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1820103.8
Basic Enclosure                                                    1554108.8
Heat Pump Water Heater                                              291290.8
Mininum Efficiency Heat Pump with Existing Heat Backup              261353.3
Heat Pump Clothes Dryer                                              83084.7
Induction Cooking                                                     4808.4
Electric Cooking                                                     -8361.1
Electric Clothes Dryer                                              -13040.1
High Efficiency Whole Home Electrification                     

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    935783.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     875249.4
Basic Enclosure + High Efficiency Whole Home Electrification       854808.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        794275.0
High Efficiency Whole Home Electrification                         618174.6
High Efficiency Heat Pump with Electric Heat Backup                353381.8
Enhanced Enclosure                                                 311413.2
Basic Enclosure                                                    249513.9
Mininum Efficiency Heat Pump with Existing Heat Backup             138334.7
Heat Pump Water Heater                                              95698.6
Heat Pump Clothes Dryer                                             16834.4
Induction Cooking                                                    1605.4
Electric Cooking                                                    -2188.9

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1192474.6
Basic Enclosure + High Efficiency Whole Home Electrification       1155230.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1133869.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1096624.9
High Efficiency Whole Home Electrification                         1002792.1
High Efficiency Heat Pump with Electric Heat Backup                 709649.5
Mininum Efficiency Heat Pump with Existing Heat Backup              398909.1
Enhanced Enclosure                                                  342380.1
Mininum Efficiency Heat Pump with Electric Heat Backup              318244.1
Basic Enclosure                                                     275923.1
Heat Pump Water Heater                                              121373.9
Mininum Efficiency Whole Home Electrification                        80347.8
Heat Pump Clothes Dryer                                        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4454559.8
Basic Enclosure + High Efficiency Whole Home Electrification       4200826.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3887241.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3633507.3
High Efficiency Whole Home Electrification                         2519136.4
Enhanced Enclosure                                                 1748982.5
Basic Enclosure                                                    1549580.8
Heat Pump Water Heater                                              569003.0
High Efficiency Heat Pump with Electric Heat Backup                 325520.1
Heat Pump Clothes Dryer                                             180582.9
Mininum Efficiency Heat Pump with Existing Heat Backup              171074.6
Induction Cooking                                                     7898.2
Electric Cooking                                               

### Annual Total Energy Savings

In [44]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_total_mmbtu'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    85121.1
Basic Enclosure + High Efficiency Whole Home Electrification       84513.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     84044.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        83437.1
High Efficiency Whole Home Electrification                         68723.3
High Efficiency Heat Pump with Electric Heat Backup                57517.6
Mininum Efficiency Whole Home Electrification                      49059.1
Enhanced Enclosure                                                 41067.6
Mininum Efficiency Heat Pump with Electric Heat Backup             40666.4
Basic Enclosure                                                    35743.9
Mininum Efficiency Heat Pump with Existing Heat Backup             13624.8
Heat Pump Water Heater                                              4809.1
Heat Pump Clothes Dryer                                              733.6
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    704485.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     697858.4
Basic Enclosure + High Efficiency Whole Home Electrification       685367.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        678741.0
High Efficiency Whole Home Electrification                         599908.0
High Efficiency Heat Pump with Electric Heat Backup                526931.9
Mininum Efficiency Whole Home Electrification                      452386.7
Mininum Efficiency Heat Pump with Electric Heat Backup             402363.3
Enhanced Enclosure                                                 270146.9
Mininum Efficiency Heat Pump with Existing Heat Backup             202575.7
Basic Enclosure                                                    202479.4
Heat Pump Water Heater                                              30322.5
Heat Pump Clothes Dryer                                              4239.1

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    75607.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74212.0
Basic Enclosure + High Efficiency Whole Home Electrification       72312.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70916.5
High Efficiency Whole Home Electrification                         62681.8
High Efficiency Heat Pump with Electric Heat Backup                49557.9
Mininum Efficiency Whole Home Electrification                      35928.8
Mininum Efficiency Heat Pump with Electric Heat Backup             32377.0
Mininum Efficiency Heat Pump with Existing Heat Backup             25583.7
Enhanced Enclosure                                                 20628.0
Basic Enclosure                                                    16173.8
Heat Pump Water Heater                                              6168.9
Induction Cooking                                                    710.8
Heat Pump Cl

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73017.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71563.6
Basic Enclosure + High Efficiency Whole Home Electrification       71501.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70047.8
High Efficiency Whole Home Electrification                         65297.4
High Efficiency Heat Pump with Electric Heat Backup                48540.7
Mininum Efficiency Heat Pump with Electric Heat Backup             32614.5
Mininum Efficiency Heat Pump with Existing Heat Backup             28262.0
Mininum Efficiency Whole Home Electrification                      27755.5
Enhanced Enclosure                                                 19814.1
Basic Enclosure                                                    15374.3
Heat Pump Water Heater                                              9355.3
Induction Cooking                                                   1011.6
Electric Coo

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    842544.4
Basic Enclosure + High Efficiency Whole Home Electrification       832217.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     825429.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        815103.0
High Efficiency Whole Home Electrification                         763776.3
High Efficiency Heat Pump with Electric Heat Backup                600303.6
Mininum Efficiency Whole Home Electrification                      450360.7
Mininum Efficiency Heat Pump with Electric Heat Backup             384542.4
Enhanced Enclosure                                                 165443.4
Basic Enclosure                                                    148946.6
Heat Pump Water Heater                                              75403.2
Mininum Efficiency Heat Pump with Existing Heat Backup              62632.7
Heat Pump Clothes Dryer                                              9891.7

### Annual Total Emission Savings

In [45]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_total_kgCO2e'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6743946.4
Basic Enclosure + High Efficiency Whole Home Electrification       6719927.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     6676037.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        6652018.2
High Efficiency Whole Home Electrification                         6117597.4
Mininum Efficiency Whole Home Electrification                      5403503.2
High Efficiency Heat Pump with Electric Heat Backup                5402664.5
Mininum Efficiency Heat Pump with Electric Heat Backup             4784192.7
Enhanced Enclosure                                                 2855133.6
Basic Enclosure                                                    2510744.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1230142.8
Heat Pump Water Heater                                              287423.4
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    55598333.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     55214842.4
Basic Enclosure + High Efficiency Whole Home Electrification       54855803.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54472312.9
High Efficiency Whole Home Electrification                         51578307.4
High Efficiency Heat Pump with Electric Heat Backup                46906561.0
Mininum Efficiency Whole Home Electrification                      46169119.5
Mininum Efficiency Heat Pump with Electric Heat Backup             42289346.0
Enhanced Enclosure                                                 18932855.1
Mininum Efficiency Heat Pump with Existing Heat Backup             16722726.8
Basic Enclosure                                                    14308996.3
Heat Pump Water Heater                                              2013962.2
Induction Cooking                                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4761913.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4680695.6
Basic Enclosure + High Efficiency Whole Home Electrification       4632881.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4551663.7
High Efficiency Whole Home Electrification                         4258248.0
High Efficiency Heat Pump with Electric Heat Backup                3493970.4
Mininum Efficiency Whole Home Electrification                      3276294.6
Mininum Efficiency Heat Pump with Electric Heat Backup             2849398.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1908306.1
Enhanced Enclosure                                                 1155530.9
Basic Enclosure                                                     895323.7
Heat Pump Water Heater                                              355179.8
Induction Cooking                                              

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4103929.1
Basic Enclosure + High Efficiency Whole Home Electrification       4046354.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4003587.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3946013.1
High Efficiency Whole Home Electrification                         3808863.6
High Efficiency Heat Pump with Electric Heat Backup                2779096.9
Mininum Efficiency Whole Home Electrification                      2419544.8
Mininum Efficiency Heat Pump with Electric Heat Backup             2178019.2
Mininum Efficiency Heat Pump with Existing Heat Backup             1630701.7
Enhanced Enclosure                                                 1021333.8
Basic Enclosure                                                     772435.0
Heat Pump Water Heater                                              606537.1
Induction Cooking                                              

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    63582112.9
Basic Enclosure + High Efficiency Whole Home Electrification       63183348.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     62497561.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        62098797.2
High Efficiency Whole Home Electrification                         60546181.4
High Efficiency Heat Pump with Electric Heat Backup                49476466.8
Mininum Efficiency Whole Home Electrification                      48981485.7
Mininum Efficiency Heat Pump with Electric Heat Backup             41358787.4
Enhanced Enclosure                                                 10610765.9
Basic Enclosure                                                     9599133.0
Heat Pump Water Heater                                              5398783.1
Mininum Efficiency Heat Pump with Existing Heat Backup              5029556.4
Induction Cooking                                  

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [46]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_mean_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                  555.1
Enhanced Enclosure + High Efficiency Whole Home Electrification     511.0
Basic Enclosure + High Efficiency Whole Home Electrification        486.6
Basic Enclosure                                                     464.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      446.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         421.7
Heat Pump Water Heater                                               95.3
Heat Pump Clothes Dryer                                              17.5
Induction Cooking                                                     1.8
Electric Cooking                                                     -5.9
Electric Clothes Dryer                                              -23.1
Mininum Efficiency Heat Pump with Existing Heat Backup              -53.5
High Efficiency Whole Home Electrification                         -148.0
High Efficiency Heat Pump

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification     635.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      575.5
Enhanced Enclosure                                                  551.0
Basic Enclosure + High Efficiency Whole Home Electrification        517.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         457.5
Basic Enclosure                                                     405.9
Heat Pump Water Heater                                               73.2
Mininum Efficiency Heat Pump with Existing Heat Backup               69.4
Heat Pump Clothes Dryer                                              21.2
Induction Cooking                                                     1.2
Electric Cooking                                                     -6.9
High Efficiency Whole Home Electrification                          -10.4
Electric Clothes Dryer                                              -19.4
High Efficiency Heat Pump

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    700.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     655.0
Basic Enclosure + High Efficiency Whole Home Electrification       639.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        594.4
High Efficiency Whole Home Electrification                         462.6
High Efficiency Heat Pump with Electric Heat Backup                264.5
Enhanced Enclosure                                                 233.1
Basic Enclosure                                                    186.7
Mininum Efficiency Heat Pump with Existing Heat Backup             103.5
Heat Pump Water Heater                                              71.6
Heat Pump Clothes Dryer                                             18.4
Induction Cooking                                                    1.2
Electric Cooking                                                    -5.4
Mininum Efficiency Whole Home Electrif

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    545.7
Basic Enclosure + High Efficiency Whole Home Electrification       528.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     518.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        501.8
High Efficiency Whole Home Electrification                         458.9
High Efficiency Heat Pump with Electric Heat Backup                324.7
Mininum Efficiency Heat Pump with Existing Heat Backup             202.6
Enhanced Enclosure                                                 199.2
Basic Enclosure                                                    169.4
Mininum Efficiency Heat Pump with Electric Heat Backup             145.6
Mininum Efficiency Whole Home Electrification                       56.0
Heat Pump Water Heater                                              55.5
Heat Pump Clothes Dryer                                             18.5
Induction Cooking                     

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    386.5
Basic Enclosure + High Efficiency Whole Home Electrification       364.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     337.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        315.3
High Efficiency Whole Home Electrification                         218.6
Enhanced Enclosure                                                 194.8
Basic Enclosure                                                    178.8
Heat Pump Water Heater                                              49.4
High Efficiency Heat Pump with Electric Heat Backup                 28.2
Heat Pump Clothes Dryer                                             20.7
Mininum Efficiency Heat Pump with Existing Heat Backup              15.3
Induction Cooking                                                    0.7
Electric Cooking                                                    -8.8
Electric Clothes Dryer                

### Annual Energy Savings Per Dwelling Unit

In [47]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_mean_mmbtu'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    139.2
Basic Enclosure + High Efficiency Whole Home Electrification       138.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     137.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        136.5
High Efficiency Whole Home Electrification                         112.4
High Efficiency Heat Pump with Electric Heat Backup                 94.1
Mininum Efficiency Whole Home Electrification                       80.2
Enhanced Enclosure                                                  67.2
Mininum Efficiency Heat Pump with Electric Heat Backup              66.5
Basic Enclosure                                                     58.5
Mininum Efficiency Heat Pump with Existing Heat Backup              22.7
Heat Pump Water Heater                                               7.9
Electric Cooking                                                     1.5
Heat Pump Clothes Dryer               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    177.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     175.4
Basic Enclosure + High Efficiency Whole Home Electrification       172.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        170.6
High Efficiency Whole Home Electrification                         150.8
High Efficiency Heat Pump with Electric Heat Backup                132.4
Mininum Efficiency Whole Home Electrification                      118.3
Mininum Efficiency Heat Pump with Electric Heat Backup             101.1
Enhanced Enclosure                                                  69.4
Mininum Efficiency Heat Pump with Existing Heat Backup              53.8
Basic Enclosure                                                     52.9
Heat Pump Water Heater                                               7.6
Electric Cooking                                                     1.5
Heat Pump Clothes Dryer               

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    56.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     55.5
Basic Enclosure + High Efficiency Whole Home Electrification       54.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        53.1
High Efficiency Whole Home Electrification                         46.9
Mininum Efficiency Whole Home Electrification                      42.1
High Efficiency Heat Pump with Electric Heat Backup                37.1
Mininum Efficiency Heat Pump with Electric Heat Backup             24.2
Mininum Efficiency Heat Pump with Existing Heat Backup             19.1
Enhanced Enclosure                                                 15.4
Basic Enclosure                                                    12.1
Heat Pump Water Heater                                              4.6
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    33.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32.7
Basic Enclosure + High Efficiency Whole Home Electrification       32.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32.1
High Efficiency Whole Home Electrification                         29.9
High Efficiency Heat Pump with Electric Heat Backup                22.2
Mininum Efficiency Whole Home Electrification                      19.3
Mininum Efficiency Heat Pump with Electric Heat Backup             14.9
Mininum Efficiency Heat Pump with Existing Heat Backup             14.4
Enhanced Enclosure                                                 11.5
Basic Enclosure                                                     9.4
Heat Pump Water Heater                                              4.3
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    73.1
Basic Enclosure + High Efficiency Whole Home Electrification       72.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     71.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        70.7
High Efficiency Whole Home Electrification                         66.3
High Efficiency Heat Pump with Electric Heat Backup                52.1
Mininum Efficiency Whole Home Electrification                      46.5
Mininum Efficiency Heat Pump with Electric Heat Backup             33.4
Enhanced Enclosure                                                 18.4
Basic Enclosure                                                    17.2
Heat Pump Water Heater                                              6.5
Mininum Efficiency Heat Pump with Existing Heat Backup              5.6
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [48]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]
saving_results[f'segment5_{income}_pct_saving_energy_mean_%'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    64.9
Basic Enclosure + High Efficiency Whole Home Electrification       64.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     64.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        63.6
High Efficiency Whole Home Electrification                         52.8
High Efficiency Heat Pump with Electric Heat Backup                43.6
Mininum Efficiency Whole Home Electrification                      37.5
Mininum Efficiency Heat Pump with Electric Heat Backup             30.8
Enhanced Enclosure                                                 30.6
Basic Enclosure                                                    25.8
Mininum Efficiency Heat Pump with Existing Heat Backup             11.0
Heat Pump Water Heater                                              4.3
Electric Cooking                                                    0.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     67.0
Basic Enclosure + High Efficiency Whole Home Electrification       66.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.5
High Efficiency Whole Home Electrification                         58.2
High Efficiency Heat Pump with Electric Heat Backup                50.5
Mininum Efficiency Whole Home Electrification                      43.8
Mininum Efficiency Heat Pump with Electric Heat Backup             37.4
Enhanced Enclosure                                                 26.4
Mininum Efficiency Heat Pump with Existing Heat Backup             21.3
Basic Enclosure                                                    19.8
Heat Pump Water Heater                                              3.4
Electric Cooking                                                    0.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.6
Basic Enclosure + High Efficiency Whole Home Electrification       57.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        56.0
High Efficiency Whole Home Electrification                         49.9
High Efficiency Heat Pump with Electric Heat Backup                39.4
Mininum Efficiency Whole Home Electrification                      37.8
Mininum Efficiency Heat Pump with Electric Heat Backup             24.6
Mininum Efficiency Heat Pump with Existing Heat Backup             21.2
Enhanced Enclosure                                                 15.1
Basic Enclosure                                                    11.6
Heat Pump Water Heater                                              5.0
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    55.6
Basic Enclosure + High Efficiency Whole Home Electrification       54.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     54.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        53.3
High Efficiency Whole Home Electrification                         50.3
High Efficiency Heat Pump with Electric Heat Backup                35.5
Mininum Efficiency Whole Home Electrification                      28.2
Mininum Efficiency Heat Pump with Existing Heat Backup             23.2
Mininum Efficiency Heat Pump with Electric Heat Backup             22.6
Enhanced Enclosure                                                 17.6
Basic Enclosure                                                    14.1
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.3
Basic Enclosure + High Efficiency Whole Home Electrification       53.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.3
High Efficiency Whole Home Electrification                         49.4
High Efficiency Heat Pump with Electric Heat Backup                35.8
Mininum Efficiency Whole Home Electrification                      29.8
Mininum Efficiency Heat Pump with Electric Heat Backup             21.4
Enhanced Enclosure                                                 12.3
Basic Enclosure                                                    11.2
Mininum Efficiency Heat Pump with Existing Heat Backup              6.8
Heat Pump Water Heater                                              6.7
Electric Cooking                                                    1.4
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [49]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_mean_kgCO2e'] = results[4]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11030.6
Basic Enclosure + High Efficiency Whole Home Electrification       10991.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     10919.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        10880.2
High Efficiency Whole Home Electrification                         10006.1
Mininum Efficiency Whole Home Electrification                       8838.1
High Efficiency Heat Pump with Electric Heat Backup                 8836.8
Mininum Efficiency Heat Pump with Electric Heat Backup              7825.2
Enhanced Enclosure                                                  4669.9
Basic Enclosure                                                     4106.7
Mininum Efficiency Heat Pump with Existing Heat Backup              2052.4
Heat Pump Water Heater                                               470.1
Electric Cooking                                                     145.9
Electric Clo

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    13974.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13877.9
Basic Enclosure + High Efficiency Whole Home Electrification       13787.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        13691.3
High Efficiency Whole Home Electrification                         12963.9
Mininum Efficiency Whole Home Electrification                      12075.2
High Efficiency Heat Pump with Electric Heat Backup                11789.7
Mininum Efficiency Heat Pump with Electric Heat Backup             10629.2
Enhanced Enclosure                                                  4862.7
Mininum Efficiency Heat Pump with Existing Heat Backup              4442.1
Basic Enclosure                                                     3737.1
Heat Pump Water Heater                                               506.2
Electric Cooking                                                     147.2
Electric Clo

segment3


upgrade_name
Mininum Efficiency Whole Home Electrification                      3838.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    3563.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3503.0
Basic Enclosure + High Efficiency Whole Home Electrification       3467.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3406.4
High Efficiency Whole Home Electrification                         3186.8
High Efficiency Heat Pump with Electric Heat Backup                2614.8
Mininum Efficiency Heat Pump with Electric Heat Backup             2132.5
Mininum Efficiency Heat Pump with Existing Heat Backup             1428.2
Enhanced Enclosure                                                  864.8
Basic Enclosure                                                     670.0
Heat Pump Water Heater                                              265.8
Electric Cooking                                                    131.6
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1877.9
Basic Enclosure + High Efficiency Whole Home Electrification       1851.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1832.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1805.7
High Efficiency Whole Home Electrification                         1742.9
Mininum Efficiency Whole Home Electrification                      1685.4
High Efficiency Heat Pump with Electric Heat Backup                1271.7
Mininum Efficiency Heat Pump with Electric Heat Backup              996.6
Mininum Efficiency Heat Pump with Existing Heat Backup              828.1
Enhanced Enclosure                                                  594.2
Basic Enclosure                                                     474.2
Heat Pump Water Heater                                              277.5
Electric Cooking                                                    135.3
Induction Cooking        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5516.6
Basic Enclosure + High Efficiency Whole Home Electrification       5482.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5422.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5387.9
High Efficiency Whole Home Electrification                         5253.2
Mininum Efficiency Whole Home Electrification                      5055.6
High Efficiency Heat Pump with Electric Heat Backup                4292.7
Mininum Efficiency Heat Pump with Electric Heat Backup             3588.4
Enhanced Enclosure                                                 1182.0
Basic Enclosure                                                    1107.3
Heat Pump Water Heater                                              468.6
Mininum Efficiency Heat Pump with Existing Heat Backup              449.2
Electric Cooking                                                    165.5
Electric Clothes Dryer   

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [50]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [51]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [52]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [53]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 5

In [54]:
segment = 'segment5'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment5_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment5_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment5_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [55]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]
saving_results[[f'segment5_{income}_energy_burden_before_upgrade_%',f'segment5_{income}_energy_burden_after_upgrade_%']] = results[4]

energy burden (%) before and after upgrade
segment1


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\i

,before,after
upgrade_name,,
Enhanced Enclosure,11.9,9.3
Basic Enclosure,11.9,9.5
Enhanced Enclosure + High Efficiency Whole Home Electrification,11.9,10.9
Basic Enclosure + High Efficiency Whole Home Electrification,11.9,10.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,11.9,11.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,11.9,11.2
Heat Pump Water Heater,11.9,11.7
Heat Pump Clothes Dryer,11.9,11.8
Induction Cooking,11.9,11.9


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,6.3,4.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,6.3,4.5
Basic Enclosure + High Efficiency Whole Home Electrification,6.3,4.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,6.3,4.7
Enhanced Enclosure,6.5,5.3
High Efficiency Whole Home Electrification,6.3,5.5
Basic Enclosure,6.5,5.7
High Efficiency Heat Pump with Electric Heat Backup,6.3,6.1
Mininum Efficiency Heat Pump with Existing Heat Backup,6.0,5.7


...

segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,9.1,5.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,9.1,5.2
Basic Enclosure + High Efficiency Whole Home Electrification,9.1,5.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,9.1,5.3
High Efficiency Whole Home Electrification,9.1,5.7
High Efficiency Heat Pump with Electric Heat Backup,9.1,6.7
Enhanced Enclosure,10.2,8.7
Mininum Efficiency Heat Pump with Existing Heat Backup,9.9,8.5
Basic Enclosure,10.6,9.4


segment5


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,5.1,3.7
Basic Enclosure + High Efficiency Whole Home Electrification,5.1,3.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,5.1,3.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,5.1,3.9
High Efficiency Whole Home Electrification,5.1,4.1
Enhanced Enclosure,5.0,4.5
Basic Enclosure,4.9,4.5
High Efficiency Heat Pump with Electric Heat Backup,5.1,4.6
Mininum Efficiency Heat Pump with Existing Heat Backup,5.1,4.9


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renter, 0-80% AMI  

Segment 2. Occupied, Single family detached, pre 1940, owner, 0-80% AMI 

Segment 3. Occupied, Multifamily with 2-4 units, 1940-1979, 0-80% AMI 

Segment 4. Occupied,  Multifamily with 5+ units, 1940-1979, 0-80% AMI

Segment 5. Occupied,  all building types, 1980+, 0-80% AMI

In [56]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_median_income = ['0-30%','30-60%','60-80%']

seg5_vintage = ['1980s', '1990s', '2000s', '2010s']
seg5_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.vintage'].isin(seg5_vintage) &
                     community_results['build_existing_model.area_median_income'].isin(seg5_median_income),
                     'segment'] = 'segment5'

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3426832256.py:29: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segment1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [57]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: segment1: 


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:9: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure,10,324.086632,392.058184,61.329412,153.928441,614.327937,795.195839
Basic Enclosure,10,324.086632,359.890586,61.329412,160.408191,614.327937,795.195839
Enhanced Enclosure + High Efficiency Whole Home Electrification,10,324.086632,136.095814,-735.092956,-26.346833,329.826487,1183.998545
Basic Enclosure + High Efficiency Whole Home Electrification,10,324.086632,136.095814,-735.092956,-26.346833,329.826487,1183.998545
Heat Pump Water Heater,10,324.086632,134.248873,-18.415492,8.539774,151.189769,453.016974
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10,324.086632,78.329087,-776.323225,-124.694194,312.632917,1144.837535
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10,324.086632,78.329087,-776.323225,-124.694194,312.632917,1144.837535
Heat Pump Clothes Dryer,10,324.086632,22.557929,0.818199,11.004019,36.335290,40.116077
Induction Cooking,10,324.086632,2.730329,-2.578596,2.962813,4.125126,5.425068


Summary for segment: segment2: 


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:9: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

...

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:9: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,48,1897.236773,566.572102,110.687299,279.327786,878.507114,1226.190301
Basic Enclosure + High Efficiency Whole Home Electrification,48,1897.236773,554.141027,70.538396,252.419700,836.553833,1226.190301
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,48,1897.236773,539.819308,65.688840,258.072700,873.966855,1143.500050
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,48,1897.236773,527.388233,63.710662,250.067495,823.022852,1143.500050
High Efficiency Whole Home Electrification,48,1897.236773,489.911959,72.473245,211.230706,773.203044,1043.926665
High Efficiency Heat Pump with Electric Heat Backup,48,1897.236773,356.545710,0.429008,69.693748,611.115614,839.453974
Mininum Efficiency Heat Pump with Existing Heat Backup,47,1681.138947,227.025056,-15.419034,27.806463,406.494572,660.720571
Enhanced Enclosure,42,1538.738451,199.340599,59.167615,73.751033,303.930775,425.866147
Mininum Efficiency Heat Pump with Electric Heat Backup,48,1897.236773,184.450588,-38.260328,-13.463458,383.989793,620.974817


Summary for segment: segment5: 


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2429053086.py:9: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,123,4678.653798,461.723981,117.544172,194.613901,606.078765,975.198896
Basic Enclosure + High Efficiency Whole Home Electrification,123,4678.653798,428.125432,75.706183,170.514862,593.440096,975.198896
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,123,4678.653798,422.280395,74.078425,154.146761,568.352667,948.110103
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,123,4678.653798,388.681846,-8.801388,124.810614,564.604758,938.115462
High Efficiency Whole Home Electrification,123,4678.653798,320.126917,-256.577602,100.461331,473.551988,885.046610
Enhanced Enclosure,95,3700.393486,171.738988,15.375151,46.522348,273.406842,434.336090
Basic Enclosure,89,3490.839281,152.786890,15.051963,42.409762,204.325219,433.376919
High Efficiency Heat Pump with Electric Heat Backup,123,4678.653798,146.045084,-444.576143,-54.239454,361.531494,675.003002
Mininum Efficiency Heat Pump with Existing Heat Backup,120,4626.495732,49.738202,-244.920786,-98.057739,140.760990,413.108030


## 3.1 Upgrade Cost Per Dwelling Unit 

In [58]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]
saving_results[f'segment5_{income}_upgrade_cost_mean_usd'] = results[4]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              3014.7
Basic Enclosure                                                     4512.8
Enhanced Enclosure                                                  5123.9
High Efficiency Heat Pump with Electric Heat Backup                28860.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32113.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32113.6
High Efficiency Whole Home Electrification                         36375.5
Basic Enclosure + High Efficiency Whole Home Electrification       36614.1
Enhanced Enclosure + High Efficiency Whole Home Electrification    36614.1
Mininum Effi

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                     1127.3
Electric Clothes Dryer                                               1266.0
Induction Cooking                                                    1953.1
Heat Pump Clothes Dryer                                              2547.5
Heat Pump Water Heater                                               3150.2
Basic Enclosure                                                      4796.8
Enhanced Enclosure                                                   6372.4
High Efficiency Heat Pump with Electric Heat Backup                 42479.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         44412.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      44440.9
Basic Enclosure + High Efficiency Whole Home Electrification        48913.0
Enhanced Enclosure + High Efficiency Whole Home Electrification     48941.4
High Efficiency Whole Home Electrification                          50129.9

segment3: upgrade cost (usd) per dwelling unit


...

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Basic Enclosure                                                      605.9
Electric Cooking                                                    1127.3
Enhanced Enclosure                                                  1767.4
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                20270.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        21699.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     22217.9
Mininum Efficiency Whole Home Electrification                      22973.4
Basic Enclosure + High Efficiency Whole Home Electrification       24047.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    24566.0
High Efficiency Whole Home Electrification                         25523.6
Mininum Effi

segment5: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Basic Enclosure                                                     1775.9
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2391.5
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              3024.5
High Efficiency Heat Pump with Electric Heat Backup                23422.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26846.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27142.9
Mininum Efficiency Whole Home Electrification                      27579.0
High Efficiency Whole Home Electrification                         29820.5
Basic Enclosure + High Efficiency Whole Home Electrification       30220.3
Enhanced Enc

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\90310082.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\90310082.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\90310082.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [59]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_per_dollar_invested_usd'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.07975
Enhanced Enclosure                                                 0.07652
Heat Pump Water Heater                                             0.04453
Heat Pump Clothes Dryer                                            0.00886
Basic Enclosure + High Efficiency Whole Home Electrification       0.00372
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00372
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00244
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00244
Induction Cooking                                                  0.00140
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00273
Electric Cooking                                                  -0.00533
High Efficiency Whole Home Electrification                        -0.01354
Electric Clothes Dryer                                            -0.01531
High Efficie

segment2


upgrade_name
Basic Enclosure                                                    0.07689
Enhanced Enclosure                                                 0.07153
Heat Pump Water Heater                                             0.02787
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01199
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01191
Basic Enclosure + High Efficiency Whole Home Electrification       0.01061
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01040
Heat Pump Clothes Dryer                                            0.00696
Induction Cooking                                                  0.00047
High Efficiency Whole Home Electrification                         0.00023
Mininum Efficiency Heat Pump with Existing Heat Backup            -0.00040
High Efficiency Heat Pump with Electric Heat Backup               -0.00527
Electric Cooking                                                  -0.00554
Mininum Effi

segment3


...

segment4


upgrade_name
Basic Enclosure                                                    0.27900
Enhanced Enclosure                                                 0.11279
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02430
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02430
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02306
Basic Enclosure + High Efficiency Whole Home Electrification       0.02304
Heat Pump Water Heater                                             0.02026
High Efficiency Whole Home Electrification                         0.01919
High Efficiency Heat Pump with Electric Heat Backup                0.01759
Heat Pump Clothes Dryer                                            0.00779
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00416
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00371
Mininum Efficiency Whole Home Electrification                      0.00342
Induction Co

segment5


upgrade_name
Basic Enclosure                                                    0.08603
Enhanced Enclosure                                                 0.07181
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01556
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01513
Heat Pump Water Heater                                             0.01498
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01448
Basic Enclosure + High Efficiency Whole Home Electrification       0.01417
High Efficiency Whole Home Electrification                         0.01074
Heat Pump Clothes Dryer                                            0.00717
High Efficiency Heat Pump with Electric Heat Backup                0.00624
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00122
Induction Cooking                                                  0.00041
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00396
Electric Coo

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\502626335.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\502626335.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\502626335.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

 ### Annual Energy Savings Per Dollar Invested

In [60]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.01281
Enhanced Enclosure                                                 0.01192
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00387
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00387
Basic Enclosure + High Efficiency Whole Home Electrification       0.00344
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00344
High Efficiency Heat Pump with Electric Heat Backup                0.00288
Heat Pump Water Heater                                             0.00282
High Efficiency Whole Home Electrification                         0.00276
Mininum Efficiency Whole Home Electrification                      0.00166
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00145
Electric Cooking                                                   0.00138
Heat Pump Clothes Dryer                                            0.00047
Electric Clo

segment2


upgrade_name
Basic Enclosure                                                    0.01008
Enhanced Enclosure                                                 0.00867
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00331
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00325
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00304
Basic Enclosure + High Efficiency Whole Home Electrification       0.00298
High Efficiency Heat Pump with Electric Heat Backup                0.00263
High Efficiency Whole Home Electrification                         0.00250
Heat Pump Water Heater                                             0.00249
Electric Cooking                                                   0.00122
Mininum Efficiency Whole Home Electrification                      0.00071
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00061
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00046
Electric Clo

segment3


...

segment4


upgrade_name
Basic Enclosure                                                    0.01468
Enhanced Enclosure                                                 0.00610
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00139
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00138
Heat Pump Water Heater                                             0.00131
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00128
Electric Cooking                                                   0.00118
High Efficiency Whole Home Electrification                         0.00111
High Efficiency Heat Pump with Electric Heat Backup                0.00104
Mininum Efficiency Whole Home Electrification                      0.00070
Heat Pump Clothes Dryer                                            0.00032
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00028
Induction Co

segment5


upgrade_name
Basic Enclosure                                                    0.00744
Enhanced Enclosure                                                 0.00609
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00196
Heat Pump Water Heater                                             0.00194
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00193
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00178
Basic Enclosure + High Efficiency Whole Home Electrification       0.00175
High Efficiency Whole Home Electrification                         0.00163
High Efficiency Heat Pump with Electric Heat Backup                0.00156
Electric Cooking                                                   0.00140
Mininum Efficiency Whole Home Electrification                      0.00129
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00053
Electric Clothes Dryer                                             0.00052
Heat Pump Cl

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2996368806.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2996368806.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2996368806.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

 ### Annual Emission Savings Per Dollar Invested

In [61]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure                                                    0.95085
Enhanced Enclosure                                                 0.87947
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.32988
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.32988
High Efficiency Heat Pump with Electric Heat Backup                0.29868
Basic Enclosure + High Efficiency Whole Home Electrification       0.29163
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.29163
High Efficiency Whole Home Electrification                         0.26683
Mininum Efficiency Whole Home Electrification                      0.20412
Mininum Efficiency Heat Pump with Electric Heat Backup             0.19470
Heat Pump Water Heater                                             0.14440
Electric Cooking                                                   0.13197
Electric Clothes Dryer                                             0.06878
Mininum Effi

segment2


upgrade_name
Basic Enclosure                                                    0.71415
Enhanced Enclosure                                                 0.60316
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.25934
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.25712
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.23732
Basic Enclosure + High Efficiency Whole Home Electrification       0.23530
High Efficiency Heat Pump with Electric Heat Backup                0.23264
High Efficiency Whole Home Electrification                         0.21370
Heat Pump Water Heater                                             0.15542
Electric Cooking                                                   0.11840
Mininum Efficiency Whole Home Electrification                      0.07686
Electric Clothes Dryer                                             0.06918
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06606
Mininum Effi

segment3


...

segment4


upgrade_name
Basic Enclosure                                                    0.70726
Enhanced Enclosure                                                 0.29995
Electric Cooking                                                   0.12138
Heat Pump Water Heater                                             0.08111
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.07355
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07269
Basic Enclosure + High Efficiency Whole Home Electrification       0.06835
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.06769
High Efficiency Whole Home Electrification                         0.06047
Mininum Efficiency Whole Home Electrification                      0.05970
High Efficiency Heat Pump with Electric Heat Backup                0.05457
Induction Cooking                                                  0.02379
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01690
Mininum Effi

segment5


upgrade_name
Basic Enclosure                                                    0.46427
Enhanced Enclosure                                                 0.37794
Heat Pump Water Heater                                             0.14015
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13796
Electric Cooking                                                   0.13777
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13751
Mininum Efficiency Whole Home Electrification                      0.13239
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12523
Basic Enclosure + High Efficiency Whole Home Electrification       0.12471
High Efficiency Whole Home Electrification                         0.12071
High Efficiency Heat Pump with Electric Heat Backup                0.11904
Electric Clothes Dryer                                             0.09415
Mininum Efficiency Heat Pump with Electric Heat Backup             0.05172
Induction Co

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3746797145.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3746797145.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3746797145.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [62]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_total_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                 127060.8
Basic Enclosure                                                    116635.7
Basic Enclosure + High Efficiency Whole Home Electrification        44106.8
Enhanced Enclosure + High Efficiency Whole Home Electrification     44106.8
Heat Pump Water Heater                                              43508.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         25385.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      25385.4
Heat Pump Clothes Dryer                                              7310.7
Induction Cooking                                                     884.9
Electric Cooking                                                     -322.3
Electric Clothes Dryer                                              -1371.2
Mininum Efficiency Heat Pump with Existing Heat Backup             -34822.4
High Efficiency Whole Home Electrification                        -159625.7

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    570093.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     514471.4
Basic Enclosure + High Efficiency Whole Home Electrification       504485.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        448863.5
Enhanced Enclosure                                                 442900.4
Basic Enclosure                                                    344289.5
Heat Pump Water Heater                                              85299.9
Heat Pump Clothes Dryer                                             17219.4
High Efficiency Whole Home Electrification                          11272.7
Induction Cooking                                                     887.0
Electric Cooking                                                    -2030.1
Electric Clothes Dryer                                              -3186.7
Mininum Efficiency Heat Pump with Existing Heat Backup             -30679.6

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1074921.4
Basic Enclosure + High Efficiency Whole Home Electrification       1051336.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1024165.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1000580.4
High Efficiency Whole Home Electrification                          929479.0
High Efficiency Heat Pump with Electric Heat Backup                 676451.6
Mininum Efficiency Heat Pump with Existing Heat Backup              381660.7
Mininum Efficiency Heat Pump with Electric Heat Backup              349946.4
Enhanced Enclosure                                                  306733.0
Basic Enclosure                                                     250993.3
Heat Pump Water Heater                                              111663.0
Mininum Efficiency Whole Home Electrification                        95732.8
Heat Pump Clothes Dryer                                        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2160246.7
Basic Enclosure + High Efficiency Whole Home Electrification       2003050.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1975703.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1818507.8
High Efficiency Whole Home Electrification                         1497763.0
High Efficiency Heat Pump with Electric Heat Backup                 683294.4
Enhanced Enclosure                                                  635501.8
Basic Enclosure                                                     533354.5
Mininum Efficiency Heat Pump with Existing Heat Backup              230113.6
Heat Pump Water Heater                                              212005.3
Heat Pump Clothes Dryer                                              47628.4
Induction Cooking                                                     3766.1
Electric Cooking                                               

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\320912423.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\320912423.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\320912423.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Annual Total Energy Savings

In [63]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_total_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       40795.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    40795.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40276.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     40276.0
High Efficiency Whole Home Electrification                         32503.8
High Efficiency Heat Pump with Electric Heat Backup                26948.2
Mininum Efficiency Whole Home Electrification                      22714.8
Enhanced Enclosure                                                 19790.1
Basic Enclosure                                                    18730.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18498.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3035.5
Heat Pump Water Heater                                              2750.5
Heat Pump Clothes Dryer                                              386.9
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    144470.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     143004.5
Basic Enclosure + High Efficiency Whole Home Electrification       141800.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        140334.4
High Efficiency Whole Home Electrification                         121727.9
High Efficiency Heat Pump with Electric Heat Backup                108351.8
Mininum Efficiency Whole Home Electrification                       82541.4
Mininum Efficiency Heat Pump with Electric Heat Backup              77811.4
Enhanced Enclosure                                                  53704.3
Basic Enclosure                                                     45130.0
Mininum Efficiency Heat Pump with Existing Heat Backup              35931.1
Heat Pump Water Heater                                               7622.1
Heat Pump Clothes Dryer                                               899.3

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59600.4
Basic Enclosure + High Efficiency Whole Home Electrification       58640.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58318.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57358.3
High Efficiency Whole Home Electrification                         53681.2
High Efficiency Heat Pump with Electric Heat Backup                39857.7
Mininum Efficiency Heat Pump with Electric Heat Backup             26572.3
Mininum Efficiency Heat Pump with Existing Heat Backup             24301.3
Mininum Efficiency Whole Home Electrification                      19749.2
Enhanced Enclosure                                                 16578.2
Basic Enclosure                                                    13202.1
Heat Pump Water Heater                                              7227.4
Induction Cooking                                                    910.8
Electric Coo

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    254242.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     248792.7
Basic Enclosure + High Efficiency Whole Home Electrification       247845.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        242395.1
High Efficiency Whole Home Electrification                         227281.1
High Efficiency Heat Pump with Electric Heat Backup                170687.6
Mininum Efficiency Whole Home Electrification                      128539.6
Mininum Efficiency Heat Pump with Electric Heat Backup             109485.9
Enhanced Enclosure                                                  53911.2
Basic Enclosure                                                     46103.7
Mininum Efficiency Heat Pump with Existing Heat Backup              35309.7
Heat Pump Water Heater                                              27521.9
Heat Pump Clothes Dryer                                              2735.9

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\4074720020.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\4074720020.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\4074720020.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Annual Total Emission Savings

In [64]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_total_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       3460473.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    3460473.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3433300.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3433300.3
High Efficiency Whole Home Electrification                         3145572.2
High Efficiency Heat Pump with Electric Heat Backup                2793601.9
Mininum Efficiency Whole Home Electrification                      2793545.1
Mininum Efficiency Heat Pump with Electric Heat Backup             2485842.5
Enhanced Enclosure                                                 1460444.3
Basic Enclosure                                                    1390657.7
Mininum Efficiency Heat Pump with Existing Heat Backup              333453.9
Heat Pump Water Heater                                              141084.6
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11285637.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11198505.5
Basic Enclosure + High Efficiency Whole Home Electrification       11182798.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11095666.5
High Efficiency Whole Home Electrification                         10409065.4
High Efficiency Heat Pump with Electric Heat Backup                 9602368.7
Mininum Efficiency Whole Home Electrification                       8971931.0
Mininum Efficiency Heat Pump with Electric Heat Backup              8469300.7
Enhanced Enclosure                                                  3734647.5
Basic Enclosure                                                     3197900.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3130076.7
Heat Pump Water Heater                                               475740.1
Induction Cooking                                  

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3154664.5
Basic Enclosure + High Efficiency Whole Home Electrification       3118411.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3064109.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3027855.8
High Efficiency Whole Home Electrification                         2928056.0
High Efficiency Heat Pump with Electric Heat Backup                2098809.7
Mininum Efficiency Whole Home Electrification                      1672433.1
Mininum Efficiency Heat Pump with Electric Heat Backup             1595898.3
Mininum Efficiency Heat Pump with Existing Heat Backup             1326731.1
Enhanced Enclosure                                                  815709.7
Basic Enclosure                                                     636263.5
Heat Pump Water Heater                                              447039.7
Induction Cooking                                              

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    17879805.0
Basic Enclosure + High Efficiency Whole Home Electrification       17632724.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     17519412.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        17272331.7
High Efficiency Whole Home Electrification                         16841317.4
Mininum Efficiency Whole Home Electrification                      13189362.3
High Efficiency Heat Pump with Electric Heat Backup                13045008.6
Mininum Efficiency Heat Pump with Electric Heat Backup             10729909.3
Enhanced Enclosure                                                  3344590.2
Basic Enclosure                                                     2878159.2
Mininum Efficiency Heat Pump with Existing Heat Backup              2569902.3
Heat Pump Water Heater                                              1983217.9
Induction Cooking                                  

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\723531023.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\723531023.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\723531023.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [65]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]
saving_results[f'segment5_{income}_saving_bill_mean_usd'] = results[4]

segment1


upgrade_name
Enhanced Enclosure                                                  392.1
Basic Enclosure                                                     359.9
Basic Enclosure + High Efficiency Whole Home Electrification        136.1
Enhanced Enclosure + High Efficiency Whole Home Electrification     136.1
Heat Pump Water Heater                                              134.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup          78.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup       78.3
Heat Pump Clothes Dryer                                              22.6
Induction Cooking                                                     2.7
Electric Cooking                                                     -6.0
Electric Clothes Dryer                                              -19.4
Mininum Efficiency Heat Pump with Existing Heat Backup             -107.4
High Efficiency Whole Home Electrification                         -492.5
High Efficiency Heat Pump

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification     586.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      529.5
Basic Enclosure + High Efficiency Whole Home Electrification        519.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         462.0
Enhanced Enclosure                                                  455.8
Basic Enclosure                                                     368.8
Heat Pump Water Heater                                               87.8
Heat Pump Clothes Dryer                                              17.7
High Efficiency Whole Home Electrification                           11.6
Induction Cooking                                                     0.9
Electric Cooking                                                     -6.2
Electric Clothes Dryer                                              -21.9
Mininum Efficiency Heat Pump with Existing Heat Backup              -34.9
High Efficiency Heat Pump

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    566.6
Basic Enclosure + High Efficiency Whole Home Electrification       554.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     539.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        527.4
High Efficiency Whole Home Electrification                         489.9
High Efficiency Heat Pump with Electric Heat Backup                356.5
Mininum Efficiency Heat Pump with Existing Heat Backup             227.0
Enhanced Enclosure                                                 199.3
Mininum Efficiency Heat Pump with Electric Heat Backup             184.5
Basic Enclosure                                                    169.1
Mininum Efficiency Whole Home Electrification                       78.5
Heat Pump Water Heater                                              58.9
Heat Pump Clothes Dryer                                             19.8
Induction Cooking                     

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    461.7
Basic Enclosure + High Efficiency Whole Home Electrification       428.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     422.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        388.7
High Efficiency Whole Home Electrification                         320.1
Enhanced Enclosure                                                 171.7
Basic Enclosure                                                    152.8
High Efficiency Heat Pump with Electric Heat Backup                146.0
Mininum Efficiency Heat Pump with Existing Heat Backup              49.7
Heat Pump Water Heater                                              45.3
Heat Pump Clothes Dryer                                             18.3
Induction Cooking                                                    0.8
Electric Cooking                                                    -8.0
Electric Clothes Dryer                

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\541036721.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\541036721.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\541036721.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Annual Energy Savings Per Dwelling Unit

In [66]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]
saving_results[f'segment5_{income}_saving_energy_mean_mmbtu'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       125.9
Enhanced Enclosure + High Efficiency Whole Home Electrification    125.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        124.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     124.3
High Efficiency Whole Home Electrification                         100.3
High Efficiency Heat Pump with Electric Heat Backup                 83.2
Mininum Efficiency Whole Home Electrification                       70.1
Enhanced Enclosure                                                  61.1
Basic Enclosure                                                     57.8
Mininum Efficiency Heat Pump with Electric Heat Backup              57.1
Mininum Efficiency Heat Pump with Existing Heat Backup               9.4
Heat Pump Water Heater                                               8.5
Electric Cooking                                                     1.6
Heat Pump Clothes Dryer               

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    148.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     147.2
Basic Enclosure + High Efficiency Whole Home Electrification       145.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        144.4
High Efficiency Whole Home Electrification                         125.3
High Efficiency Heat Pump with Electric Heat Backup                111.5
Mininum Efficiency Whole Home Electrification                       93.8
Mininum Efficiency Heat Pump with Electric Heat Backup              80.1
Enhanced Enclosure                                                  55.3
Basic Enclosure                                                     48.3
Mininum Efficiency Heat Pump with Existing Heat Backup              40.9
Heat Pump Water Heater                                               7.8
Electric Cooking                                                     1.4
Heat Pump Clothes Dryer               

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    31.4
Basic Enclosure + High Efficiency Whole Home Electrification       30.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     30.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30.2
High Efficiency Whole Home Electrification                         28.3
High Efficiency Heat Pump with Electric Heat Backup                21.0
Mininum Efficiency Whole Home Electrification                      16.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14.5
Mininum Efficiency Heat Pump with Electric Heat Backup             14.0
Enhanced Enclosure                                                 10.8
Basic Enclosure                                                     8.9
Heat Pump Water Heater                                              3.8
Electric Cooking                                                    1.3
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.2
Basic Enclosure + High Efficiency Whole Home Electrification       53.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.8
High Efficiency Whole Home Electrification                         48.6
High Efficiency Heat Pump with Electric Heat Backup                36.5
Mininum Efficiency Whole Home Electrification                      35.6
Mininum Efficiency Heat Pump with Electric Heat Backup             23.4
Enhanced Enclosure                                                 14.6
Basic Enclosure                                                    13.2
Mininum Efficiency Heat Pump with Existing Heat Backup              7.6
Heat Pump Water Heater                                              5.9
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\927876618.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\927876618.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\927876618.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Annual Percentage Energy Savings Per Dwelling Unit

In [67]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]
saving_results[f'segment5_{income}_pct_saving_energy_mean_%'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       61.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    61.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        60.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60.8
High Efficiency Whole Home Electrification                         48.5
High Efficiency Heat Pump with Electric Heat Backup                38.8
Mininum Efficiency Whole Home Electrification                      33.7
Enhanced Enclosure                                                 29.4
Basic Enclosure                                                    26.8
Mininum Efficiency Heat Pump with Electric Heat Backup             26.0
Heat Pump Water Heater                                              5.4
Mininum Efficiency Heat Pump with Existing Heat Backup              3.0
Electric Cooking                                                    0.9
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    67.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     66.7
Basic Enclosure + High Efficiency Whole Home Electrification       66.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        65.9
High Efficiency Whole Home Electrification                         57.8
High Efficiency Heat Pump with Electric Heat Backup                50.4
Mininum Efficiency Whole Home Electrification                      42.7
Mininum Efficiency Heat Pump with Electric Heat Backup             36.1
Enhanced Enclosure                                                 24.7
Basic Enclosure                                                    20.7
Mininum Efficiency Heat Pump with Existing Heat Backup             20.5
Heat Pump Water Heater                                              4.2
Electric Cooking                                                    0.8
Heat Pump Clothes Dryer                            

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    54.7
Basic Enclosure + High Efficiency Whole Home Electrification       53.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     53.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        52.6
High Efficiency Whole Home Electrification                         49.7
High Efficiency Heat Pump with Electric Heat Backup                34.8
Mininum Efficiency Whole Home Electrification                      25.0
Mininum Efficiency Heat Pump with Existing Heat Backup             23.8
Mininum Efficiency Heat Pump with Electric Heat Backup             21.8
Enhanced Enclosure                                                 17.3
Basic Enclosure                                                    13.8
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    2.4
Heat Pump Clothes Dryer                            

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    57.3
Basic Enclosure + High Efficiency Whole Home Electrification       56.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     56.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54.9
High Efficiency Whole Home Electrification                         52.0
High Efficiency Heat Pump with Electric Heat Backup                36.5
Mininum Efficiency Whole Home Electrification                      35.0
Mininum Efficiency Heat Pump with Electric Heat Backup             23.0
Enhanced Enclosure                                                 13.3
Basic Enclosure                                                    11.6
Mininum Efficiency Heat Pump with Existing Heat Backup             10.1
Heat Pump Water Heater                                              7.6
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1754187417.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1754187417.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1754187417.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Annual Emission Savings Per Dwelling Unit

In [68]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]
saving_results[f'segment5_{income}_saving_emission_mean_kgCO2e'] = results[4]

segment1


upgrade_name
Basic Enclosure + High Efficiency Whole Home Electrification       10677.6
Enhanced Enclosure + High Efficiency Whole Home Electrification    10677.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        10593.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     10593.8
High Efficiency Whole Home Electrification                          9706.0
High Efficiency Heat Pump with Electric Heat Backup                 8619.9
Mininum Efficiency Whole Home Electrification                       8619.7
Mininum Efficiency Heat Pump with Electric Heat Backup              7670.3
Enhanced Enclosure                                                  4506.3
Basic Enclosure                                                     4291.0
Mininum Efficiency Heat Pump with Existing Heat Backup              1028.9
Heat Pump Water Heater                                               435.3
Electric Cooking                                                     148.8
Electric Clo

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11614.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     11525.2
Basic Enclosure + High Efficiency Whole Home Electrification       11509.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11419.4
High Efficiency Whole Home Electrification                         10712.8
Mininum Efficiency Whole Home Electrification                      10194.8
High Efficiency Heat Pump with Electric Heat Backup                 9882.5
Mininum Efficiency Heat Pump with Electric Heat Backup              8716.4
Enhanced Enclosure                                                  3843.6
Mininum Efficiency Heat Pump with Existing Heat Backup              3562.7
Basic Enclosure                                                     3425.7
Heat Pump Water Heater                                               489.6
Electric Cooking                                                     133.5
Electric Clo

segment3


...

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1662.8
Basic Enclosure + High Efficiency Whole Home Electrification       1643.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1615.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1595.9
High Efficiency Whole Home Electrification                         1543.3
Mininum Efficiency Whole Home Electrification                      1371.4
High Efficiency Heat Pump with Electric Heat Backup                1106.2
Mininum Efficiency Heat Pump with Electric Heat Backup              841.2
Mininum Efficiency Heat Pump with Existing Heat Backup              789.2
Enhanced Enclosure                                                  530.1
Basic Enclosure                                                     428.5
Heat Pump Water Heater                                              235.6
Electric Cooking                                                    136.8
Induction Cooking        

segment5


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3821.6
Basic Enclosure + High Efficiency Whole Home Electrification       3768.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3744.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3691.7
Mininum Efficiency Whole Home Electrification                      3651.3
High Efficiency Whole Home Electrification                         3599.6
High Efficiency Heat Pump with Electric Heat Backup                2788.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2293.4
Enhanced Enclosure                                                  903.8
Basic Enclosure                                                     824.5
Mininum Efficiency Heat Pump with Existing Heat Backup              555.5
Heat Pump Water Heater                                              423.9
Electric Cooking                                                    155.3
Electric Clothes Dryer   

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3614653602.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3614653602.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3614653602.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [69]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

### Segment 2

In [70]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Segment 3

In [71]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Segment 4

In [72]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

### Segment 5

In [73]:
segment = 'segment5'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment5_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment5_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment5_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\1874328925.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [74]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]
saving_results[[f'segment5_{income}_energy_burden_before_upgrade_%',f'segment5_{income}_energy_burden_after_upgrade_%']] = results[4]

energy burden (%) before and after upgrade
segment1


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\2591935474.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\1\i

,before,after
upgrade_name,,
Enhanced Enclosure,19.5,15.3
Basic Enclosure,19.5,15.5
Basic Enclosure + High Efficiency Whole Home Electrification,19.5,18.5
Enhanced Enclosure + High Efficiency Whole Home Electrification,19.5,18.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,19.5,18.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,19.5,18.9
Heat Pump Water Heater,19.5,19.2
Heat Pump Clothes Dryer,19.5,19.3
Induction Cooking,19.5,19.5


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,16.5,10.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,16.5,11.1
Basic Enclosure + High Efficiency Whole Home Electrification,16.5,11.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,16.5,11.6
High Efficiency Whole Home Electrification,16.5,13.6
Enhanced Enclosure,16.5,14.0
Basic Enclosure,17.0,15.1
High Efficiency Heat Pump with Electric Heat Backup,16.5,15.0
Mininum Efficiency Heat Pump with Existing Heat Backup,15.8,15.2


...

segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.1,5.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,5.7
Basic Enclosure + High Efficiency Whole Home Electrification,10.1,5.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.1,5.9
High Efficiency Whole Home Electrification,10.1,6.3
High Efficiency Heat Pump with Electric Heat Backup,10.1,7.3
Mininum Efficiency Heat Pump with Existing Heat Backup,11.2,9.5
Enhanced Enclosure,11.1,9.4
Basic Enclosure,11.3,10.0


segment5


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,9.5,6.5
Basic Enclosure + High Efficiency Whole Home Electrification,9.5,6.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,9.5,6.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,9.5,7.0
High Efficiency Whole Home Electrification,9.5,7.3
High Efficiency Heat Pump with Electric Heat Backup,9.5,8.4
Enhanced Enclosure,9.1,8.2
Basic Enclosure,9.0,8.2
Mininum Efficiency Heat Pump with Existing Heat Backup,9.4,9.0


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_17980\3291905064.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

In [75]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [77]:
! jupyter nbconvert --output-dir='./results/duluth/' --to=html --no-input saving_results_reports_for_duluth.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\traitlets\traitlets.py:2202: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './results/duluth/' instead of "'./results/duluth/'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook saving_results_reports_for_duluth.ipynb to html
[NbConvertApp] Writing 4828922 bytes to results\duluth\saving_results_reports_for_duluth.html
